In [17]:
import numpy as np
import pandas as pd
import re

1. 평점 (Rating): `rating`
2. 아이디 (User ID): `user_id`
3. 작성날짜 (Review Date): `review_date`
4. 제품명 (Product Name): `product_name`
5. 간단리뷰 (Brief Review): `user_comment`
6. 리뷰유형 (Review Type): `review_type`
    - 이 변수는 '한달사용기', '재구매', 'BEST' 등의 값을 포함할 수 있다.
7. 리뷰내용 (Review Content): `review_content`
8. 추천수 (Number of Likes/Recommendations): `recommendation_count`

## FootHealth

In [18]:
df = pd.read_excel('data/FootHealth.xlsx')
df.head()

,review_date,user_id,rating,product_info,review_text,image_yn,recommendation_count
0,24.01.02.,ccm5***,5,교정 단계: 2단계 / 제품 모델: 액티브(DFH801) / 사이즈: 270~280...,아들이 군대갈때 평발이 있어서 가져갔는데 훈련받을때도 발이 아프지 않고 편하다고 하...,Y,0
1,24.01.02.,kore*****,5,교정 단계: 2단계 / 제품 모델: 액티런(DFH803) / 사이즈: 225~235...,평발이라 조금만 걸어도 발이 아파서 ㅠㅠ기능성 깔창을 계속 사용해온 1인 입니다 여...,N,0
2,24.01.01.,imju******,5,교정 단계: 2단계 / 제품 모델: 액티런(DFH803) / 사이즈: 270~280...,일때문에 자주 걷는편인데\n자꾸 아프게 되어 여러가지 알아보다\n구매했습니다\n일단...,N,0
3,23.12.31.,sept*********,5,교정 단계: 2단계 / 제품 모델: 액티브(DFH801) / 사이즈: 225~235...,제가 진짜 좋다는 깔창은 나 사서 써본 사람으로써 이제품 찐으로 좋아요 ㅠㅠ \n하...,N,0
4,23.12.31.,dmsq********,5,교정 단계: 2단계 / 제품 모델: 액티브(DFH801) / 사이즈: 210~220...,한달사용기몇년째 군화에 넣고 활용중인데 좋아요. 가격이 좀 비싸서 타회사 다른 상품...,N,0


In [19]:
df.shape

(6380, 7)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6380 entries, 0 to 6379
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   review_date           6380 non-null   object
 1   user_id               6380 non-null   object
 2   rating                6380 non-null   int64 
 3   product_info          6380 non-null   object
 4   review_text           6380 non-null   object
 5   image_yn              6380 non-null   object
 6   recommendation_count  6380 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 349.0+ KB


In [4]:
# product_info -> product_name, user_comment 등 분리
df.product_info[0]

'교정 단계: 2단계 / 제품 모델: 액티브(DFH801) / 사이즈: 270~280\n마감처리깔끔해요사이즈정사이즈예요'

In [5]:
def split_product_info(df, brand_name):
    product_step = []
    product_name = []
    product_size = []
    user_comment = []

    if brand_name == '닥터풋헬스': # - 교정 단계, 제품 모델, 사이즈
        for txt in df.product_info:
            splited_txt = txt.split('\n')
            if len(splited_txt) == 2: # 제품정보 + 사용자 코멘트 둘 다 있는 경우
                product = splited_txt[0].split(' / ')
                product_step.append(product[0].split(': ')[-1])
                product_name.append(product[1].split(': ')[-1])
                try: # 사이즈를 입력하지 않은 경우 대비
                    product_size.append(product[2].split(': ')[-1])
                except:
                    product_size.append(np.nan)

                user_comment.append(splited_txt[1])
            else: # 사용자 고멘트만 있는 경우
                product_step.append(np.nan)
                product_name.append(np.nan)
                product_size.append(np.nan)
                user_comment.append(splited_txt[0])
            
        df['product_step'] = product_step
        df['product_name'] = product_name
        df['product_size'] = product_size
        df['user_comment'] = user_comment

    elif brand_name == '레드닥터': # 사이즈
        for txt in df.product_info:
            splited_txt = txt.split('\n')
            product = splited_txt[0].split(' / ')
            product_size.append(product[0].split(': ')[-1])

            user_comment.append(splited_txt[1])
    elif brand_name == '엑스솔': # 제품. 컬러, 사이즈
        for txt in df.product_info:
            splited_txt = txt.split('\n')
            product = splited_txt[0].split(' / ')
            # product_step.append(product[0].split(': ')[-1])
            product_name.append(product[0].split(': ')[-1])
            product_size.append(product[2].split(': ')[-1])

            user_comment.append(splited_txt[1])
    
    return df

In [6]:
prep_df = split_product_info(df, '닥터풋헬스')

- 평소사이즈 -  mm `usual_size_mm`
- 발유형 - 칼발, 평발, 땀 많은 발, `foot_type`
- 마감처리 - 아주 깔끔해요, 깔끔해요, 엉성해요 `finish_qualtiy`
- 사이즈 - 정사이즈예요, 생각보다 커요, 작아요 `size_fit`

In [9]:
# 초기화
prep_df['usual_size_mm'] = np.nan
prep_df['narrow_feet'] = False
prep_df['flat_feet'] = False
prep_df['sweaty_feet'] = False
prep_df['finish_quality'] = np.nan
prep_df['size_fit'] = np.nan

# product_info 내용을 각 항목별로 분리
def process_comment(comment):
    sizes = ['210', '215', '220', '225', '230', '235', '240', '245', '250', '255', '260', '265', '270', '275', '280', '285', '290', '295']
    finish_qualities = ['깔끔해', '아주 깔끔해', '엉성해']
    size_fits = ['정', '생각보다 커', '작아']

    splited_comment = re.split("평소사이즈|mm|발|마감처리|예요|요|사이즈", comment)
    filtered_list = [item for item in splited_comment if item != '']  # 빈 문자열 요소 제거

    result = {}
    for e in filtered_list:
        if e in sizes:
            result['usual_size_mm'] = e
        elif e == '칼':
            result['narrow_feet'] = True
        elif e == '평':
            result['flat_feet'] = True
        elif e == '땀 많은 ':
            result['sweaty_feet'] = True
        elif e in finish_qualities:
            result['finish_quality'] = e
        elif e in size_fits:
            result['size_fit'] = e
    return pd.Series(result)

# 각 row에 해당 function 적용
prep_df.update(prep_df['user_comment'].apply(process_comment))

In [10]:
# 날짜형식으로 변환
prep_df['review_date'] = pd.to_datetime(prep_df['review_date'], format="%y.%m.%d.")

In [11]:
# NaN 초기화
prep_df['is_best'] = False
prep_df['is_one_month'] = False
prep_df['is_repurchase'] = False

# 세 가지 종류의 리뷰 유형은 중복 가능, 단어는 '한달사용기' > 'BEST' > '재구매' 순서로 나타남.
# '한달사용기' prefix 확인 후 해당 단어 제거
one_month_mask = prep_df['review_text'].str.startswith('한달사용기')
prep_df.loc[one_month_mask, 'is_one_month'] = True
prep_df.loc[one_month_mask, 'review_text'] = prep_df.loc[one_month_mask, 'review_text'].str.replace('한달사용기', '', 1)

# 'BEST' prefix 확인 후 해당 단어 제거
best_mask = prep_df['review_text'].str.startswith('BEST')
prep_df.loc[best_mask, 'is_best'] = True
prep_df.loc[one_month_mask, 'review_text'] = prep_df.loc[one_month_mask, 'review_text'].str.replace('BEST', '', 1)

# Check for '재구매' prefix and update columns
repurchase_mask = prep_df['review_text'].str.startswith('재구매')
prep_df.loc[repurchase_mask, 'is_repurchase'] = True
prep_df.loc[one_month_mask, 'review_text'] = prep_df.loc[one_month_mask, 'review_text'].str.replace('재구매', '', 1)

In [12]:
prep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6380 entries, 0 to 6379
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   review_date           6380 non-null   datetime64[ns]
 1   user_id               6380 non-null   object        
 2   rating                6380 non-null   int64         
 3   product_info          6380 non-null   object        
 4   review_text           6380 non-null   object        
 5   image_yn              6380 non-null   object        
 6   recommendation_count  6380 non-null   int64         
 7   product_step          6055 non-null   object        
 8   product_name          6055 non-null   object        
 9   product_size          6033 non-null   object        
 10  user_comment          6380 non-null   object        
 11  usual_size_mm         282 non-null    object        
 12  narrow_feet           6380 non-null   object        
 13  flat_feet         

In [14]:
prep_df = prep_df.drop(['product_info', 'user_comment'], axis=1)

In [20]:
prep_df.finish_quality.unique()

array(['깔끔해', '아주 깔끔해', '엉성해'], dtype=object)

In [21]:
prep_df.size_fit.unique()

array(['정', '생각보다 커', '작아'], dtype=object)

- finish_quality
    - '아주 깔끔해': 'Very Good'
    - '깔끔해': 'Good'
    - '엉성해': 'Poor'

- size_fit
    - '정': 'Regular'
    - '생각보다 커': 'Larger than Expected'
    - '작아': 'Smaller than Expected'

In [22]:
# 'finish_quality' 컬럼의 값 변경
prep_df['finish_quality'] = prep_df['finish_quality'].replace({
                                                                '아주 깔끔해': 'Very Good',
                                                                '깔끔해': 'Good',
                                                                '엉성해': 'Poor'
                                                            })

# 'size_fit' 컬럼의 값 변경
prep_df['size_fit'] = prep_df['size_fit'].replace({
                                                    '정': 'Regular',
                                                    '생각보다 커': 'Larger than Expected',
                                                    '작아': 'Smaller than Expected'
                                                })

In [23]:
prep_df.head()

,review_date,user_id,rating,review_text,image_yn,recommendation_count,product_step,product_name,product_size,usual_size_mm,narrow_feet,flat_feet,sweaty_feet,finish_quality,size_fit,is_best,is_one_month,is_repurchase
0,2024-01-02,ccm5***,5,아들이 군대갈때 평발이 있어서 가져갔는데 훈련받을때도 발이 아프지 않고 편하다고 하...,Y,0,2단계,액티브(DFH801),270~280,NaN,False,False,False,Good,Regular,False,False,False
1,2024-01-02,kore*****,5,평발이라 조금만 걸어도 발이 아파서 ㅠㅠ기능성 깔창을 계속 사용해온 1인 입니다 여...,N,0,2단계,액티런(DFH803),225~235,NaN,False,False,False,Very Good,Regular,False,False,False
2,2024-01-01,imju******,5,일때문에 자주 걷는편인데\n자꾸 아프게 되어 여러가지 알아보다\n구매했습니다\n일단...,N,0,2단계,액티런(DFH803),270~280,NaN,False,False,False,Very Good,Regular,False,False,False
3,2023-12-31,sept*********,5,제가 진짜 좋다는 깔창은 나 사서 써본 사람으로써 이제품 찐으로 좋아요 ㅠㅠ \n하...,N,0,2단계,액티브(DFH801),225~235,235,True,False,False,Very Good,Regular,False,False,False
4,2023-12-31,dmsq********,5,몇년째 군화에 넣고 활용중인데 좋아요. 가격이 좀 비싸서 타회사 다른 상품도 몇번 ...,N,0,2단계,액티브(DFH801),210~220,NaN,False,False,False,Very Good,Regular,False,True,False


In [24]:
prep_df.to_excel("data/prep_data/prep_FootHealth.xlsx", index=False)